In [1]:
%pip install -q pandas networkx matplotlib requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd, networkx as nx, requests, matplotlib
print(pd.__version__, nx.__version__, requests.__version__, matplotlib.__version__)

2.1.4 3.1 2.28.1 3.8.0


In [3]:
from getpass import getpass
API_KEY = getpass("WMATA API key: ")

import requests
url = "https://api.wmata.com/gtfs/rail-gtfs-static.zip"
r = requests.get(url, headers={"api_key": API_KEY}, timeout=60)
r.raise_for_status()
open("rail_gtfs.zip","wb").write(r.content)
print("Saved rail_gtfs.zip")

WMATA API key:  ········


Saved rail_gtfs.zip


In [5]:
import zipfile, os

ZIP = "rail_gtfs.zip" 
assert os.path.exists(ZIP), "rail_gtfs.zip not found in this folder."

with zipfile.ZipFile(ZIP) as z:
    print("Files in GTFS:", sorted(z.namelist())[:10])

Files in GTFS: ['agency.txt', 'calendar_dates.txt', 'feed_info.txt', 'levels.txt', 'pathways.txt', 'routes.txt', 'shapes.txt', 'stop_times.txt', 'stops.txt', 'trips.txt']


In [9]:
import pandas as pd, zipfile, io, networkx as nx

ZIP = "rail_gtfs.zip"

with zipfile.ZipFile(ZIP) as z:
    stops = pd.read_csv(io.TextIOWrapper(z.open("stops.txt"), encoding="utf-8"))
    stime = pd.read_csv(io.TextIOWrapper(z.open("stop_times.txt"), encoding="utf-8"))[
        ["trip_id","stop_id","stop_sequence"]
    ]

# roll platforms up to parent stations so nodes = stations (not platforms)
stops["station_id"] = stops.apply(
    lambda r: r["parent_station"] if pd.notna(r.get("parent_station")) else r["stop_id"], axis=1
)
name_by_station = (stops.dropna(subset=["station_id"])
                   .drop_duplicates(subset=["station_id"])
                   .set_index("station_id")["stop_name"])
stop_to_station = stops.set_index("stop_id")["station_id"].to_dict()

# undirected edges between consecutive stations on each trip
pairs = []
for _, seg in stime.sort_values(["trip_id","stop_sequence"]).groupby("trip_id"):
    sids = [stop_to_station[s] for s in seg["stop_id"] if s in stop_to_station]
    for a, b in zip(sids, sids[1:]):
        if a != b:
            pairs.append(tuple(sorted((a, b))))

# dedupe and build graph
G = nx.Graph()
for u, v in pd.Series(pairs).drop_duplicates():
    G.add_edge(u, v)

bet = nx.betweenness_centrality(G, normalized=True, weight=None)
cent = (pd.Series(bet).rename_axis("station_id").reset_index(name="betweenness"))
cent["station_name"] = cent["station_id"].map(name_by_station)
cent = cent[["station_name","betweenness"]].dropna()
cent.sort_values("betweenness", ascending=False).head(10)

,station_name,betweenness
11,L'ENFANT PLAZA,0.550974
62,GALLERY PLACE METRORAIL STATION,0.399842
20,PENTAGON METRORAIL STATION,0.365335
18,ROSSLYN METRORAIL STATION,0.332617
14,METRO CENTER METRORAIL STATION,0.329109
44,COURT HOUSE METRORAIL STATION,0.305412
43,CLARENDON METRORAIL STATION,0.292096
42,VIRGINIA SQ-GMU METRORAIL STATION,0.278351
63,ARCHIVES METRORAIL STATION,0.272265
41,BALLSTON-MU METRORAIL STATION,0.264175


In [1]:
import pandas as pd
YEAR = 2024
RID_FILE = "ridership_export.csv"  

rid = pd.read_csv(RID_FILE, low_memory=False)
rid.columns = rid.columns.str.lower().str.strip().str.replace(r"\s+","_", regex=True)

# pick station column and one numeric entries measure
station_col = "station_name" if "station_name" in rid.columns else ("station" if "station" in rid.columns else None)
assert station_col, f"No station column found. Columns: {rid.columns.tolist()}"

if "entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["entries"], errors="coerce")
elif {"tap_entries","nontapped_entries"} <= set(rid.columns):
    rid["entries_val"] = (pd.to_numeric(rid["tap_entries"], errors="coerce").fillna(0) +
                          pd.to_numeric(rid["nontapped_entries"], errors="coerce").fillna(0))
else:
    raise ValueError("Need 'Entries' or 'Tap Entries' + 'NonTapped Entries' in your export.")

# belt-and-suspenders filters if present
if "date" in rid.columns:
    rid["date"] = pd.to_datetime(rid["date"], errors="coerce")
    rid = rid[rid["date"].between(f"{YEAR}-01-01", f"{YEAR}-12-31")]
if "day_of_week" in rid.columns:
    wk = {"mon","tue","wed","thu","fri"}
    rid = rid[rid["day_of_week"].astype(str).str.lower().str[:3].isin(wk)]

# collapse time-of-day to all-day totals, then average across weekdays
group = [station_col]
if "date" in rid.columns: group += ["date"]
if "time_period" in rid.columns: group += ["time_period"]

day_sum = rid.groupby(group, as_index=False)["entries_val"].sum()
by_station = (day_sum.groupby(station_col, as_index=False)["entries_val"]
              .mean().rename(columns={"entries_val": "avg_weekday_entries_2024"}))

# tidy name column
by_station = by_station.rename(columns={station_col: "station_name"})
by_station.head()

C:\Users\Arsh\AppData\Local\Temp\ipykernel_67184\2217431899.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  rid["date"] = pd.to_datetime(rid["date"], errors="coerce")


,station_name,avg_weekday_entries_2024
0,Addison Road,329.992469
1,Anacostia,854.751464
2,Archives,1055.102929
3,Arlington Cemetery,201.408476
4,Ashburn,285.724686


In [3]:
# === Build "2024 average weekday entries by station" from ridership_export.csv ===
import pandas as pd
YEAR = 2024
RID_FILE = "ridership_export.csv"   # leave as-is if your file is named this

# Robust read (Tableau exports can be utf-16)
def read_tableau_csv(p):
    try:
        return pd.read_csv(p, low_memory=False)
    except UnicodeError:
        return pd.read_csv(p, encoding="utf-16", engine="python", low_memory=False)

rid = read_tableau_csv(RID_FILE)
rid.columns = rid.columns.str.lower().str.strip().str.replace(r"\s+","_", regex=True)

# pick columns present in your file
station_col = "station_name" if "station_name" in rid.columns else ("station" if "station" in rid.columns else None)
assert station_col, f"No 'Station Name' column found. Got: {rid.columns.tolist()}"

# build a single numeric entries value
if "entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["entries"], errors="coerce")
elif {"tap_entries","nontapped_entries"} <= set(rid.columns):
    rid["entries_val"] = (pd.to_numeric(rid["tap_entries"], errors="coerce").fillna(0) +
                          pd.to_numeric(rid["nontapped_entries"], errors="coerce").fillna(0))
elif "avg_daily_entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["avg_daily_entries"], errors="coerce")
else:
    raise ValueError("Need 'Entries' or 'Tap Entries' + 'NonTapped Entries' (or 'Avg Daily Entries').")

# belt-and-suspenders filters if present
if "date" in rid.columns:
    # set the exact format if you know it; else let pandas infer:
    try:
        rid["date"] = pd.to_datetime(rid["date"], format="%m/%d/%Y", errors="coerce")
    except Exception:
        rid["date"] = pd.to_datetime(rid["date"], errors="coerce")
    rid = rid[rid["date"].between(f"{YEAR}-01-01", f"{YEAR}-12-31")]

if "day_of_week" in rid.columns:
    wk = {"mon","tue","wed","thu","fri"}
    rid = rid[rid["day_of_week"].astype(str).str.lower().str[:3].isin(wk)]

# collapse any time-of-day to all-day totals, then average across days
group = [station_col]
if "date" in rid.columns: group.append("date")
if "time_period" in rid.columns: group.append("time_period")

day_sum = rid.groupby(group, as_index=False)["entries_val"].sum()
by_station = (day_sum.groupby(station_col, as_index=False)["entries_val"]
              .mean()
              .rename(columns={station_col: "station_name", "entries_val": "avg_weekday_entries_2024"}))

by_station = by_station.sort_values("avg_weekday_entries_2024", ascending=False)
print("Stations:", len(by_station))
display(by_station.head(10))

by_station.to_csv("wmata_ridership_by_station_2024.csv", index=False)
print("Saved -> wmata_ridership_by_station_2024.csv")

Stations: 0


,station_name,avg_weekday_entries_2024


Saved -> wmata_ridership_by_station_2024.csv


In [9]:
import numpy as np, pandas as pd, matplotlib.pyplot as plt
cent = pd.read_csv("wmata_betweenness.csv")   # columns: station_name, betweenness
rid  = pd.read_csv("wmata_ridership_by_station_2024.csv")

# light name normalization
def norm(s):
    return (str(s).upper().replace("’","'").strip())
cent["jn"] = cent["station_name"].map(norm)
rid["jn"]  = rid["station_name"].map(norm)

df = cent.merge(rid, on="jn", suffixes=("_cent","_rid"))
df["station_name"] = df["station_name_cent"]

# log–log residuals
eps = 1e-9
X = np.log(df["avg_weekday_entries_2024"] + eps)
Y = np.log(df["betweenness"] + eps)
b0, b1 = np.polyfit(X, Y, 1)
df["residual"] = Y - (b0 + b1*X)

top_hidden  = df.sort_values("residual", ascending=False).head(10)[
    ["station_name","betweenness","avg_weekday_entries_2024","residual"]
]
top_crowded = df.sort_values("residual", ascending=True).head(10)[
    ["station_name","betweenness","avg_weekday_entries_2024","residual"]
]

display(top_hidden.head(5))
display(top_crowded.head(5))
top_hidden.to_csv("top_hidden_chokepoints.csv", index=False)
top_crowded.to_csv("top_crowded_not_central.csv", index=False)

# quick figure
plt.figure(figsize=(7,5))
plt.scatter(X, Y, alpha=0.5)
xg = np.linspace(X.min(), X.max(), 200)
plt.plot(xg, b0 + b1*xg)
plt.xlabel("log(avg weekday entries 2024)")
plt.ylabel("log(betweenness)")
plt.title("WMATA: centrality vs ridership (log–log)")
plt.tight_layout(); plt.show()


TypeError: expected non-empty vector for x

In [7]:
# Compute station betweenness from WMATA GTFS and save to wmata_betweenness.csv
import os, io, zipfile, pandas as pd, networkx as nx

ZIP = "rail_gtfs.zip"  # change if yours has a different name
assert os.path.exists(ZIP), f"{ZIP} not found in {os.getcwd()}"

with zipfile.ZipFile(ZIP) as z:
    # read GTFS core files
    stops = pd.read_csv(io.TextIOWrapper(z.open("stops.txt"), encoding="utf-8"))
    stime = pd.read_csv(io.TextIOWrapper(z.open("stop_times.txt"), encoding="utf-8"))[
        ["trip_id","stop_id","stop_sequence"]
    ]

# ---- roll platforms up so nodes = stations (not platforms) ----
# If parent_station exists, use it; otherwise, treat stop_id as its own station
if "parent_station" in stops.columns:
    stops["station_id"] = stops.apply(
        lambda r: r["parent_station"] if pd.notna(r.get("parent_station")) and str(r["parent_station"]).strip() != "" 
        else r["stop_id"], axis=1
    )
else:
    stops["station_id"] = stops["stop_id"]

# Pick one name per station_id (prefer rows marked as stations if present)
if "location_type" in stops.columns:
    # location_type==1 are stations; 0 are platforms
    name_df = (stops.sort_values(["station_id","location_type"], ascending=[True, False])
                     .drop_duplicates(subset=["station_id"]))
else:
    name_df = stops.drop_duplicates(subset=["station_id"])
name_by_station = name_df.set_index("station_id")["stop_name"]

# Map stop_id -> station_id for edge building
stop_to_station = stops.set_index("stop_id")["station_id"].to_dict()

# ---- build undirected edges between consecutive stations on each trip ----
pairs = []
for _, seg in stime.sort_values(["trip_id","stop_sequence"]).groupby("trip_id"):
    sids = [stop_to_station[s] for s in seg["stop_id"] if s in stop_to_station]
    for a, b in zip(sids, sids[1:]):
        if a != b:
            pairs.append(tuple(sorted((a, b))))

# dedupe edges
edges = pd.Series(pairs).drop_duplicates().tolist()

# ---- graph + betweenness ----
G = nx.Graph()
G.add_nodes_from(name_by_station.index)
G.add_edges_from(edges)

bet = nx.betweenness_centrality(G, normalized=True, weight=None)

cent = (pd.Series(bet, name="betweenness")
          .rename_axis("station_id")
          .reset_index())
cent["station_name"] = cent["station_id"].map(name_by_station)
cent = cent[["station_name","betweenness"]].dropna()

cent.sort_values("betweenness", ascending=False).head(10)
cent.to_csv("wmata_betweenness.csv", index=False)
print("Saved -> wmata_betweenness.csv")
print("Stations in graph:", G.number_of_nodes(), "Edges:", G.number_of_edges())

Saved -> wmata_betweenness.csv
Stations in graph: 98 Edges: 103


In [11]:
import pandas as pd

cent = pd.read_csv("wmata_betweenness.csv")              # from GTFS step
rid  = pd.read_csv("wmata_ridership_by_station_2024.csv")# from your export aggregation

print("GTFS stations:", cent.shape[0], "Ridership stations:", rid.shape[0])

print("\nSample GTFS names:", cent['station_name'].dropna().sample(8, random_state=0).tolist())
print("Sample ridership names:", rid['station_name'].dropna().sample(8, random_state=1).tolist())


GTFS stations: 98 Ridership stations: 0

Sample GTFS names: ['NOMA-GALLAUDET U', 'DUNN LORING', 'DUPONT CIRCLE', 'SHAW-HOWARD U', 'SOUTHERN AVE', 'BALLSTON-MU', 'JUDICIARY SQUARE', 'FRANCONIA SPRINGFIELD']


ValueError: a must be greater than 0 unless no samples are taken

In [13]:
import pandas as pd

# robust reader (handles Tableau CSV encodings)
def read_tableau_csv(p):
    try:
        return pd.read_csv(p, low_memory=False)
    except UnicodeError:
        return pd.read_csv(p, encoding="utf-16", engine="python", low_memory=False)

raw = read_tableau_csv("ridership_export.csv")
print("raw shape:", raw.shape)
print("raw columns:", list(raw.columns))
display(raw.head(5))

raw shape: (755547, 13)
raw columns: ['Year of Date', 'Date', 'Day of Week', 'Holiday', 'Service Type', 'Station Name', 'Time Period', 'Year', 'Avg Daily Tapped Entries', 'Entries', 'NonTapped Entries', 'SUM([NonTapped Entries])/COUNTD([Date])', 'Tap Entries']


,Year of Date,Date,Day of Week,Holiday,Service Type,Station Name,Time Period,Year,Avg Daily Tapped Entries,Entries,NonTapped Entries,SUM([NonTapped Entries])/COUNTD([Date]),Tap Entries
0,2024,2/17/2024 12:00:00 AM,Sat,No,Saturday,Tysons,AM Peak (Open-9:30am),2024,0,2,0,0,2
1,2024,5/25/2024 12:00:00 AM,Sat,No,Saturday,Court House,AM Peak (Open-9:30am),2024,0,0,0,0,0
2,2024,6/29/2024 12:00:00 AM,Sat,No,Saturday,Arlington Cemetery,AM Peak (Open-9:30am),2024,0,0,0,0,0
3,2024,1/27/2024 12:00:00 AM,Sat,No,Saturday,Van Ness-UDC,AM Peak (Open-9:30am),2024,0,0,0,0,0
4,2024,8/31/2024 12:00:00 AM,Sat,No,Saturday,Pentagon,AM Peak (Open-9:30am),2024,0,0,0,0,0


In [15]:
rid = raw.copy()
rid.columns = rid.columns.str.lower().str.strip().str.replace(r"\s+","_", regex=True)

print("normalized columns:", list(rid.columns))
print("has station_name?", "station_name" in rid.columns or "station" in rid.columns)

# What numeric measures do we have?
cands = [c for c in rid.columns if any(k in c for k in
    ["entries","tap","nontapped","non_tapped","avg_daily"])]
print("numeric-ish columns:", cands)

normalized columns: ['year_of_date', 'date', 'day_of_week', 'holiday', 'service_type', 'station_name', 'time_period', 'year', 'avg_daily_tapped_entries', 'entries', 'nontapped_entries', 'sum([nontapped_entries])/countd([date])', 'tap_entries']
has station_name? True
numeric-ish columns: ['avg_daily_tapped_entries', 'entries', 'nontapped_entries', 'sum([nontapped_entries])/countd([date])', 'tap_entries']


In [17]:
import numpy as np

# pick station column
station_col = "station_name" if "station_name" in rid.columns else ("station" if "station" in rid.columns else None)
assert station_col, f"No station column found. Got: {rid.columns.tolist()}"

# build a single numeric entries column
if "entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["entries"], errors="coerce")
elif {"tap_entries","nontapped_entries"} <= set(rid.columns):
    rid["entries_val"] = (pd.to_numeric(rid["tap_entries"], errors="coerce").fillna(0) +
                          pd.to_numeric(rid["nontapped_entries"], errors="coerce").fillna(0))
elif "avg_daily_entries" in rid.columns:
    # use only if it’s clearly non-zero
    rid["entries_val"] = pd.to_numeric(rid["avg_daily_entries"], errors="coerce")
else:
    raise ValueError("Need 'Entries' or 'Tap Entries' + 'NonTapped Entries' (or non-zero 'Avg Daily Entries').")

# optional: filter to 2024 only *IF* a date column exists
if "date" in rid.columns:
    # try common WMATA formats first; fall back to infer
    try:
        rid["date"] = pd.to_datetime(rid["date"], format="%m/%d/%Y", errors="coerce")
    except Exception:
        rid["date"] = pd.to_datetime(rid["date"], errors="coerce")
    print("date min/max:", rid["date"].min(), rid["date"].max())
    # keep 2024 only — but only if we actually have 2024 rows
    has_2024 = (rid["date"].dt.year == 2024).any()
    if has_2024:
        rid = rid[(rid["date"] >= "2024-01-01") & (rid["date"] <= "2024-12-31")]
    else:
        print("⚠️ No 2024 dates detected; skipping the 2024 filter so we don't drop everything.")

# optional: weekdays only if a day-of-week column exists
for dow in ["day_of_week","weekday","day_type"]:
    if dow in rid.columns:
        wk = {"mon","tue","wed","thu","fri"}
        rid = rid[rid[dow].astype(str).str.lower().str[:3].isin(wk)]
        break

# collapse time-of-day to daily totals if a time_period column exists
group = [station_col]
if "date" in rid.columns: group.append("date")
if "time_period" in rid.columns: group.append("time_period")

day_sum = rid.groupby(group, as_index=False)["entries_val"].sum()

# now average to station level
by_station = (day_sum.groupby(station_col, as_index=False)["entries_val"]
              .mean()
              .rename(columns={station_col:"station_name", "entries_val":"avg_weekday_entries_2024"}))

print("stations in by_station:", len(by_station))
display(by_station.head(10))

# save for the join step
by_station.to_csv("wmata_ridership_by_station_2024.csv", index=False)

date min/max: NaT NaT
⚠️ No 2024 dates detected; skipping the 2024 filter so we don't drop everything.
stations in by_station: 0


,station_name,avg_weekday_entries_2024


In [19]:
import pandas as pd, numpy as np, re

RID_FILE = "ridership_export.csv"  # change if your filename is different

def read_tableau_csv(p):
    try:
        return pd.read_csv(p, low_memory=False)
    except UnicodeError:
        return pd.read_csv(p, encoding="utf-16", engine="python", low_memory=False)

rid = read_tableau_csv(RID_FILE)
rid.columns = rid.columns.str.lower().str.strip().str.replace(r"\s+","_", regex=True)

# --- pick station column ---
station_col = "station_name" if "station_name" in rid.columns else ("station" if "station" in rid.columns else None)
assert station_col, f"No station column found. Got: {rid.columns.tolist()}"

# --- build numeric entries value ---
if "entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["entries"], errors="coerce")
elif {"tap_entries","nontapped_entries"} <= set(rid.columns):
    rid["entries_val"] = (pd.to_numeric(rid["tap_entries"], errors="coerce").fillna(0) +
                          pd.to_numeric(rid["nontapped_entries"], errors="coerce").fillna(0))
elif "avg_daily_entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["avg_daily_entries"], errors="coerce")
else:
    raise ValueError("Need 'Entries' or 'Tap Entries' + 'NonTapped Entries' (or non-zero 'Avg Daily Entries').")

# --- parse dates if present; choose best year automatically ---
date_col = "date" if "date" in rid.columns else None
chosen_year = None

if date_col:
    # Try common WMATA formats first; fall back to infer
    try:
        rid[date_col] = pd.to_datetime(rid[date_col], format="%m/%d/%Y", errors="coerce")
    except Exception:
        rid[date_col] = pd.to_datetime(rid[date_col], errors="coerce")

    years = rid[date_col].dt.year.dropna().astype(int)
    year_counts = years.value_counts().sort_index()
    print("Years in file:\n", year_counts)

    if (years == 2024).any():
        chosen_year = 2024
        print("Using preferred year 2024.")
    else:
        chosen_year = int(years.max()) if len(years) else None
        print(f"⚠️ No 2024 rows detected. Using latest available year: {chosen_year}")

    if chosen_year is not None:
        rid = rid[rid[date_col].dt.year == chosen_year]

    # Weekdays: use provided day_of_week if present, else derive from date
    if "day_of_week" in rid.columns:
        wk = {"mon","tue","wed","thu","fri"}
        rid = rid[rid["day_of_week"].astype(str).str.lower().str[:3].isin(wk)]
    else:
        rid = rid[rid[date_col].dt.dayofweek <= 4]  # 0=Mon ... 4=Fri

# --- collapse time-of-day to daily totals, then average by station ---
group = [station_col]
if date_col: group.append(date_col)
if "time_period" in rid.columns and date_col:
    day_sum = rid.groupby([station_col, date_col], as_index=False)["entries_val"].sum()
elif date_col:
    day_sum = rid[[station_col, date_col, "entries_val"]].copy()
else:
    # No dates: assume export already reflects your intended period; sum across time_period if present
    if "time_period" in rid.columns:
        day_sum = rid.groupby(station_col, as_index=False)["entries_val"].sum().assign(dummy_date=pd.NaT)
        date_col = "dummy_date"
    else:
        day_sum = rid.groupby(station_col, as_index=False)["entries_val"].mean().assign(dummy_date=pd.NaT)
        date_col = "dummy_date"

by_station = (day_sum.groupby(station_col, as_index=False)["entries_val"]
              .mean()
              .rename(columns={station_col: "station_name", "entries_val": "avg_weekday_entries_2024"}))

print("Stations aggregated:", len(by_station))
display(by_station.head(10))

# Save under the expected name (even if the year used wasn't 2024)
by_station.to_csv("wmata_ridership_by_station_2024.csv", index=False)
print("Saved -> wmata_ridership_by_station_2024.csv (if not truly 2024, this still holds your chosen year).")


Years in file:
 Series([], Name: count, dtype: int64)
⚠️ No 2024 rows detected. Using latest available year: None
Stations aggregated: 0


,station_name,avg_weekday_entries_2024


Saved -> wmata_ridership_by_station_2024.csv (if not truly 2024, this still holds your chosen year).


In [23]:
# === Bulletproof ridership loader -> station averages ===
import pandas as pd, numpy as np, re

RID_FILE = "ridership_export.csv"  # change if your filename differs

def read_tableau_csv(p):
    # Try common encodings Tableau uses
    for enc in [None, "utf-16", "utf-8-sig"]:
        try:
            return pd.read_csv(p, encoding=enc, engine="python", low_memory=False) if enc else pd.read_csv(p, low_memory=False)
        except Exception as e:
            last_e = e
    raise last_e

rid = read_tableau_csv(RID_FILE)
rid.columns = rid.columns.str.lower().str.strip().str.replace(r"\s+","_", regex=True)

# 1) Station column
station_col = "station_name" if "station_name" in rid.columns else ("station" if "station" in rid.columns else None)
assert station_col, f"No station column found. Columns: {list(rid.columns)}"

# 2) Numeric entries column
if "entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["entries"], errors="coerce")
elif {"tap_entries","nontapped_entries"} <= set(rid.columns):
    rid["entries_val"] = (pd.to_numeric(rid["tap_entries"], errors="coerce").fillna(0) +
                          pd.to_numeric(rid["nontapped_entries"], errors="coerce").fillna(0))
elif "avg_daily_entries" in rid.columns:
    rid["entries_val"] = pd.to_numeric(rid["avg_daily_entries"], errors="coerce")
else:
    raise ValueError("Need 'Entries' or 'Tap Entries' + 'NonTapped Entries' (or non-zero 'Avg Daily Entries').")

# 3) Dates: try to use them; otherwise ignore gracefully
date_col = "date" if "date" in rid.columns else None
used_year = None
if date_col:
    # Try several formats, then infer
    parsed = None
    for fmt in ["%m/%d/%Y", "%Y-%m-%d", "%m/%d/%Y %I:%M %p", "%Y-%m-%d %H:%M:%S"]:
        try:
            parsed = pd.to_datetime(rid[date_col], format=fmt, errors="coerce")
            if parsed.notna().mean() > 0.5:
                break
        except Exception:
            parsed = None
    if parsed is None or parsed.notna().mean() <= 0.2:
        print("⚠️ Date column not parseable in a consistent way; ignoring dates and proceeding.")
        date_col = None
    else:
        rid[date_col] = parsed
        counts = rid[date_col].dt.year.value_counts().sort_index()
        print("Years present:", counts.to_dict())
        if (rid[date_col].dt.year == 2024).any():
            used_year = 2024
        else:
            used_year = int(counts.index.max())
            print(f"⚠️ No 2024 rows; using latest available year: {used_year}")
        rid = rid[rid[date_col].dt.year == used_year]
        # Weekdays: use provided day_of_week if present, else derive
        if "day_of_week" in rid.columns:
            wk = {"mon","tue","wed","thu","fri"}
            rid = rid[rid["day_of_week"].astype(str).str.lower().str[:3].isin(wk)]
        else:
            rid = rid[rid[date_col].dt.dayofweek <= 4]

# 4) Collapse time-of-day if needed, then average by station
group = [station_col]
if date_col: group.append(date_col)
if "time_period" in rid.columns and date_col:
    day_sum = rid.groupby([station_col, date_col], as_index=False)["entries_val"].sum()
elif date_col:
    day_sum = rid[[station_col, date_col, "entries_val"]].copy()
else:
    # No usable dates: aggregate to station totals over whatever period the export represents
    if "time_period" in rid.columns:
        day_sum = rid.groupby(station_col, as_index=False)["entries_val"].sum().assign(dummy_date=pd.NaT)
        date_col = "dummy_date"
    else:
        day_sum = rid.groupby(station_col, as_index=False)["entries_val"].mean().assign(dummy_date=pd.NaT)
        date_col = "dummy_date"

by_station = (day_sum.groupby(station_col, as_index=False)["entries_val"]
              .mean()
              .rename(columns={station_col: "station_name", "entries_val": "avg_weekday_entries_2024"}))

print("✅ Stations aggregated:", len(by_station))
display(by_station.head(10))

by_station.to_csv("wmata_ridership_by_station_2024.csv", index=False)
print("Saved -> wmata_ridership_by_station_2024.csv",
      "(label says 2024 even if dates were missing; it contains the period in your export).")


⚠️ Date column not parseable in a consistent way; ignoring dates and proceeding.
✅ Stations aggregated: 98


,station_name,avg_weekday_entries_2024
0,Addison Road,465856.0
1,Anacostia,1206700.0
2,Archives,1574083.0
3,Arlington Cemetery,346356.0
4,Ashburn,431765.0
5,Ballston-MU,1807365.0
6,Benning Road,597249.0
7,Bethesda,1535317.0
8,Braddock Road,715322.0
9,Branch Ave,775906.0


Saved -> wmata_ridership_by_station_2024.csv (label says 2024 even if dates were missing; it contains the period in your export).


In [29]:
import pandas as pd, numpy as np, re, matplotlib.pyplot as plt
from pathlib import Path

# Inputs created earlier
cent = pd.read_csv("wmata_betweenness.csv")                # station_name, betweenness
rid  = pd.read_csv("wmata_ridership_by_station_2024_daily.csv")  # station_name, avg_weekday_entries_2024

# Light name normalization so sources match
RENAME = {
    "WHITE FLINT": "NORTH BETHESDA",
    "PRINCE GEORGE'S PLAZA": "HYATTSVILLE CROSSING",
    "PRINCE GEORGES PLAZA": "HYATTSVILLE CROSSING",
    "KING ST-OLD TOWN": "KING STREET-OLD TOWN",
    "U STREET/AFRICAN-AMER CIVIL WAR MEMORIAL/CARDOZO": "U STREET",
    "NOMA-GALLAUDET U": "NOMA – GALLAUDET U",   # add/adjust if your ridership has typographic variants
}
def norm(s: str) -> str:
    if not isinstance(s, str): return ""
    s = s.upper().strip().replace("’","'").replace("&","AND").replace("/"," ").replace("–","-")
    s = re.sub(r"[^A-Z0-9\- ]"," ", s)
    s = re.sub(r"\s+"," ", s).strip()
    return RENAME.get(s, s)

cent["jn"] = cent["station_name"].map(norm)
rid["jn"]  = rid["station_name"].map(norm)

df = cent.merge(rid, on="jn", how="inner", suffixes=("_cent","_rid")).copy()
df["station_name"] = df["station_name_cent"]

# Drop zeros (log-space)
eps = 1e-9
df = df[(df["betweenness"] > 0) & (df["avg_weekday_entries_2024"] > 0)]

# Fit log–log trend: betweenness ~ f(entries)
X = np.log(df["avg_weekday_entries_2024"] + eps)
Y = np.log(df["betweenness"] + eps)
b0, b1 = np.polyfit(X, Y, 1)
df["residual"] = Y - (b0 + b1*X)  # + = more central than entries predict

# Top lists you’ll report
top_hidden  = df.sort_values("residual", ascending=False).head(10)[
    ["station_name","betweenness","avg_weekday_entries_2024","residual"]
]
top_crowded = df.sort_values("residual", ascending=True).head(10)[
    ["station_name","betweenness","avg_weekday_entries_2024","residual"]
]

Path("figs").mkdir(exist_ok=True)
top_hidden.to_csv("top_hidden_chokepoints.csv", index=False)
top_crowded.to_csv("top_crowded_not_central.csv", index=False)

# Scatter + fit line
plt.figure(figsize=(7,5))
plt.scatter(X, Y, alpha=0.5)
xg = np.linspace(X.min(), X.max(), 200); plt.plot(xg, b0 + b1*xg)
plt.xlabel("log(avg weekday entries)"); plt.ylabel("log(betweenness)")
plt.title("WMATA: centrality vs ridership (log–log)")
plt.tight_layout(); plt.savefig("figs/centrality_vs_ridership.png", dpi=200); plt.close()

# Horizontal bar chart of “punch above weight”
plt.figure(figsize=(8,5))
(top_hidden.assign(name=lambda d: d["station_name"].str.slice(0,24))
           .sort_values("residual")
           .plot(kind="barh", x="name", y="residual", legend=False,
                 title="Stations that punch above their weight"))
plt.xlabel("Positive residual (more central than entries predict)")
plt.tight_layout(); plt.savefig("figs/punch_above_weight.png", dpi=200); plt.close()

print("✅ Saved: top_hidden_chokepoints.csv, top_crowded_not_central.csv,",
      "figs/centrality_vs_ridership.png, figs/punch_above_weight.png")
top_hidden.head(5)


✅ Saved: top_hidden_chokepoints.csv, top_crowded_not_central.csv, figs/centrality_vs_ridership.png, figs/punch_above_weight.png


,station_name,betweenness,avg_weekday_entries_2024,residual
0,METRO CENTER,0.329109,15767.152672,73.133477
15,UNION STATION,0.137887,16750.645038,72.732414
34,L'ENFANT PLAZA,0.550974,12083.973282,71.587076
2,DUPONT CIRCLE,0.219072,13117.790076,71.300927
1,FARRAGUT NORTH,0.234536,11654.278626,70.452422


<Figure size 800x500 with 0 Axes>

In [27]:
import pandas as pd
rid = pd.read_csv("wmata_ridership_by_station_2024.csv")
# if numbers look huge, treat them as year totals and divide by weekday count
weekday_days_2024 = pd.date_range("2024-01-01", "2024-12-31", freq="B").size  # Mon–Fri ≈ 262 in 2024
rid["avg_weekday_entries_2024"] = rid["avg_weekday_entries_2024"] / weekday_days_2024
rid.to_csv("wmata_ridership_by_station_2024_daily.csv", index=False)
print("Saved -> wmata_ridership_by_station_2024_daily.csv")

Saved -> wmata_ridership_by_station_2024_daily.csv


In [31]:
import os, glob
print("Here:", os.getcwd())
print("PNGs:", glob.glob("figs/*.png"))
print("CSVs:", glob.glob("*.csv"))


Here: C:\Users\Arsh\Downloads
PNGs: ['figs\\centrality_vs_ridership.png', 'figs\\damage_rate_dca_bwi.png', 'figs\\monthly_trend_dca_bwi.png', 'figs\\monthly_trend_rolling.png', 'figs\\punch_above_weight.png', 'figs\\top_species_damaging_dca_bwi.png']
CSVs: ['apple_quality.csv', 'Arsh-IAM-Admin_credentials (1).csv', 'Arsh-IAM-Admin_credentials.csv', 'arsh.malik_accessKeys.csv', 'arsh.malik_credentials.csv', 'CrashData_test_6478750435646127290.csv', 'customers.csv', 'Data Range (1).csv', 'Data Range.csv', 'diabetes.csv', 'insurance.csv', 'line_items.csv', 'MillionSongsFinal.csv', 'orders.csv', 'Primary.csv', 'products.csv', 'results.csv', 'results2.csv', 'ridership_export.csv', 'Social_Network_Ads.csv', 'top_crowded_not_central.csv', 'top_hidden_chokepoints.csv', 'Tweets.csv', 'wmata_betweenness.csv', 'wmata_ridership_by_station_2024.csv', 'wmata_ridership_by_station_2024_daily.csv']


In [33]:
import pandas as pd, matplotlib.pyplot as plt

top_crowded = (pd.read_csv("top_crowded_not_central.csv")
                 .sort_values("residual", ascending=True)
                 .head(10))

plt.figure(figsize=(8,5))
plt.barh(top_crowded["station_name"][::-1], (-top_crowded["residual"])[::-1])
plt.xlabel("Magnitude of negative residual")
plt.title("Crowded but not central")
plt.tight_layout()
plt.savefig("figs/crowded_not_central.png", dpi=200)
plt.close()

print("Saved -> figs/crowded_not_central.png")

Saved -> figs/crowded_not_central.png
